<img src="yelp_logo.png" width="400">

# Analyzing Yelp Reviews

### Import libraries

In [1]:
from pyspark.sql import SparkSession
import pyspark
from pyspark import sql
import re
import pandas as pd
import numpy as np

### Create Spark session

In [2]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

### Load Data

#### Dataset available at: https://www.kaggle.com/yelp-dataset/yelp-dataset

In [3]:
path = '/Users/ryanfrench/Dropbox/cloud_documents/college/9_fall_2019/IST_718/yelp_dataset/yelp_academic_dataset_review.json'
reviews_df = spark.read.json(path)
reviews_df.count()

6685900

# REMEMBER TO REMOVE THIS, THIS SAMPLES ONLY 1/1000 OF THE DATA!

In [4]:
reviews_df = reviews_df.sample(False, 0.001, seed = 123)

### Check schema

In [5]:
# Check schema
reviews_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



### Check number of NULL values per column

In [6]:
for col in reviews_df.columns:
    print(col, ':', reviews_df.filter(reviews_df[col].isNull()).count())

business_id : 0
cool : 0
date : 0
funny : 0
review_id : 0
stars : 0
text : 0
useful : 0
user_id : 0


### View data

In [7]:
reviews_df.show(5)
reviews_df.count()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|hcFSc0OHgZJybnjQB...|   0|2014-02-05 18:10:23|    0|4CMCRoNZgmxwtOBLG...|  5.0|Fresher noodles y...|     0|5BfyObsvVjB6zcj8W...|
|HhVmDybpU7L50Kb5A...|   0|2017-06-06 19:51:49|    0|QdXaCGCQtvSiue-0m...|  2.0|I would not be go...|     0|zPByj_3y6TBok5BpC...|
|XXCIO_TXCE1uQWPLr...|   1|2015-03-04 19:51:08|    1|uqAUjIy1OKnNl1rzJ...|  1.0|Don't waste your ...|     1|EZxdmp1_mXzPaHreZ...|
|Hpi9raHlFm6prTSzF...|   1|2018-07-07 21:56:54|    1|UldTdufMzctB0llHL...|  2.0|On a second visit...|     2|moj7DUHkP4j9yfwFm...|
|cisqOxAgQPjVVr9Yz...|   0|2018-07-26 14:25:15|    0|ylZjXOIZZqZPBRztX...|  4.0|I absolute

6585

## Can we predict high star Yelp reviews based off of their text content?

### By creating a text transformation and prediction pipeline can we predict which Yelp reviews will have a high number of stars (4/5 or above)?

### Create dummy variable for high reviews (defined as a 4/5 or above)

#### By creating a dummy variable we can easily predict the binary outcomes of low vs high reviews.

In [8]:
from pyspark.sql import Row
from pyspark.sql.functions import col, when

In [9]:
reviews_df = reviews_df.withColumn('d_stars',when(col('stars')>=4, 1).otherwise(0))
reviews_df = reviews_df.select('d_stars', 'text')
reviews_df.groupBy('d_stars').count().show()

+-------+-----+
|d_stars|count|
+-------+-----+
|      1| 4341|
|      0| 2244|
+-------+-----+



In [10]:
reviews_df.show(5)

+-------+--------------------+
|d_stars|                text|
+-------+--------------------+
|      1|Fresher noodles y...|
|      0|I would not be go...|
|      0|Don't waste your ...|
|      0|On a second visit...|
|      1|I absolutely love...|
+-------+--------------------+
only showing top 5 rows



### Tokenize 'text' column

#### By tokenizing the 'text' column we can break out the long strings of review text into their individual words.

In [11]:
from pyspark.ml.feature import Tokenizer

In [12]:
tokenizer = Tokenizer().setInputCol('text').setOutputCol('tokens')

#### Sample output of Tokenizer

In [13]:
tokenizer.transform(reviews_df).show()

+-------+--------------------+--------------------+
|d_stars|                text|              tokens|
+-------+--------------------+--------------------+
|      1|Fresher noodles y...|[fresher, noodles...|
|      0|I would not be go...|[i, would, not, b...|
|      0|Don't waste your ...|[don't, waste, yo...|
|      0|On a second visit...|[on, a, second, v...|
|      1|I absolutely love...|[i, absolutely, l...|
|      0|This place USED t...|[this, place, use...|
|      0|Beware. My dog wa...|[beware., my, dog...|
|      1|First time here w...|[first, time, her...|
|      1|Plumbing Update w...|[plumbing, update...|
|      1|I frequent this p...|[i, frequent, thi...|
|      1|This is my second...|[this, is, my, se...|
|      0|The front desk st...|[the, front, desk...|
|      1|Great service, sm...|[great, service,,...|
|      1|Love this place! ...|[love, this, plac...|
|      1|This is so much f...|[this, is, so, mu...|
|      1|The pollo mole is...|[the, pollo, mole...|
|      1|We 

### Get NTLK stop words from GitHub

#### Stop words are words which are generally considered to offer little insight in linguistic processing due to the fact that they are utilized so frequently. We will be utilizing the list contained in the Natural Language Tool Kit (NLTK) to remove these words in this case.

In [14]:
import requests

stop_words = requests.get('https://gist.github.com/sebleier/554280').text.split()

### Remove stop words

In [15]:
from pyspark.ml.feature import StopWordsRemover

sw_filter = StopWordsRemover(). \
            setStopWords(stop_words). \
            setCaseSensitive(False). \
            setInputCol("tokens"). \
            setOutputCol("filtered_tokens")

# ADJUST THE MINDF PARAM WITH FULL REVIEWS TAKEN INTO ACCOUNT

### Create CountVectorizer Estimator

#### The CountVectorizer counts the number of times that a token appears in each review and then saves this list of numbers as a vector for each review.

In [16]:
from pyspark.ml.feature import CountVectorizer

# we will remove words that appear in 50 docs or less
cv = CountVectorizer(minTF=1., minDF=6, vocabSize=2**17). \
     setInputCol("filtered_tokens"). \
     setOutputCol("tf")

### Create initial text processing pipeline

In [17]:
from pyspark.ml import Pipeline

cv_pipeline = Pipeline(stages=[tokenizer, sw_filter, cv]).fit(reviews_df)

#### Sample output of initial text processing pipeline

In [18]:
cv_pipeline.transform(reviews_df).show(5)

+-------+--------------------+--------------------+--------------------+--------------------+
|d_stars|                text|              tokens|     filtered_tokens|                  tf|
+-------+--------------------+--------------------+--------------------+--------------------+
|      1|Fresher noodles y...|[fresher, noodles...|[fresher, noodles...|(6567,[3,8,11,17,...|
|      0|I would not be go...|[i, would, not, b...|[back.., personal...|(6567,[0,4,14,18,...|
|      0|Don't waste your ...|[don't, waste, yo...|[waste, money., 5...|(6567,[0,8,25,37,...|
|      0|On a second visit...|[on, a, second, v...|[visit,, changed....|(6567,[0,6,7,8,54...|
|      1|I absolutely love...|[i, absolutely, l...|[absolutely, chat...|(6567,[0,1,3,7,9,...|
+-------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



### Create Inverse Document Frequency (IDF) stage for the pipeline

#### This will lower the realtive value of tokens which appear frequently in documents allowing those that are more specialized to stand out more.

In [19]:
from pyspark.ml.feature import IDF

idf = IDF(). \
      setInputCol('tf'). \
      setOutputCol('tfidf')

### Create IDF text processing pipeline

In [20]:
idf_pipeline = Pipeline(stages=[cv_pipeline, idf]).fit(reviews_df)

#### Sample output of IDF text processing pipeline

In [21]:
idf_pipeline.transform(reviews_df).show(5)

+-------+--------------------+--------------------+--------------------+--------------------+--------------------+
|d_stars|                text|              tokens|     filtered_tokens|                  tf|               tfidf|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      1|Fresher noodles y...|[fresher, noodles...|[fresher, noodles...|(6567,[3,8,11,17,...|(6567,[3,8,11,17,...|
|      0|I would not be go...|[i, would, not, b...|[back.., personal...|(6567,[0,4,14,18,...|(6567,[0,4,14,18,...|
|      0|Don't waste your ...|[don't, waste, yo...|[waste, money., 5...|(6567,[0,8,25,37,...|(6567,[0,8,25,37,...|
|      0|On a second visit...|[on, a, second, v...|[visit,, changed....|(6567,[0,6,7,8,54...|(6567,[0,6,7,8,54...|
|      1|I absolutely love...|[i, absolutely, l...|[absolutely, chat...|(6567,[0,1,3,7,9,...|(6567,[0,1,3,7,9,...|
+-------+--------------------+--------------------+--------------------+--------

### Create Text Frequency Inverse Document Frequency (TFIDF) Reviews Dataframe

#### By passing the reviews data frame to the IDF pipeline we can create a TFIDF dataframe which we will then use to predict the review scores.

In [22]:
tfidf_df = idf_pipeline.transform(reviews_df)
pd_tfidf_df = tfidf_df.toPandas()

In [23]:
tfidf_df.columns

['d_stars', 'text', 'tokens', 'filtered_tokens', 'tf', 'tfidf']

In [24]:
lr_df = pd_tfidf_df[['d_stars', 'tfidf']]
lr_df.head()

,d_stars,tfidf
0,1,"(0.0, 0.0, 0.0, 1.3560841977020832, 0.0, 0.0, ..."
1,0,"(2.9074505057164317, 0.0, 0.0, 0.0, 1.41744568..."
2,0,"(1.1629802022865727, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0,"(3.4889406068597184, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1,"(1.1629802022865727, 1.328191628299782, 0.0, 1..."


### Create prediction pipeline for estimating high star reviews

### Split data into training, validation, and testing groups

In [25]:
train_df, validation_df, test_df = reviews_df.randomSplit([0.6, 0.3, 0.1], seed=0)

#### Number of training records

In [26]:
train_df.count()

3942

#### Number of validation records

In [27]:
validation_df.count()

2017

#### Number of testing records

In [28]:
test_df.count()

626

### Build logistic regression model for later predictions

In [29]:
from pyspark.ml.classification import LogisticRegression

In [30]:
en_lr = LogisticRegression(). \
     setLabelCol('d_stars'). \
     setFeaturesCol('tfidf'). \
     setRegParam(0.0). \
     setMaxIter(100). \
     setElasticNetParam(0.)

### Create new pipeline for prediction

In [31]:
en_lr_estimator = Pipeline(stages=[tokenizer, 
                                   sw_filter, 
                                   cv, 
                                   idf, 
                                   en_lr])

### Build grid search estimator pipeline

#### This will allow us to iteratively search for the optimal $\lambda$ and $\alpha$ parameters to achieve the highest accuracy.

In [32]:
from pyspark.ml.tuning import ParamGridBuilder

In [33]:
en_lr_estimator.getStages()

[Tokenizer_44dbea8f2808,
 StopWordsRemover_b51a0393d1ab,
 CountVectorizer_758f5f2a3056,
 IDF_ab3e96ba3323,
 LogisticRegression_adde6b5d10b3]

Lets create a pipeline transformation by chaining the `idf_pipeline` with the logistic regression step (`lr`)

In [34]:
grid = ParamGridBuilder(). \
       addGrid(en_lr.regParam, [0., 0.01, 0.02]). \
       addGrid(en_lr.elasticNetParam, [0., 0.2, 0.4]). \
       build()

#### View grid parameters

In [35]:
grid

[{Param(parent='LogisticRegression_adde6b5d10b3', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
  Param(parent='LogisticRegression_adde6b5d10b3', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0},
 {Param(parent='LogisticRegression_adde6b5d10b3', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
  Param(parent='LogisticRegression_adde6b5d10b3', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.2},
 {Param(parent='LogisticRegression_adde6b5d10b3', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
  Param(parent='LogisticRegression_adde6b5d10b3', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.4},
 {Pa

In [36]:
all_models = []

for j in range(len(grid)):
    print("Fitting model {}".format(j+1))
    model = en_lr_estimator.fit(train_df, grid[j])
    all_models.append(model)

Fitting model 1
Fitting model 2
Fitting model 3
Fitting model 4
Fitting model 5
Fitting model 6
Fitting model 7
Fitting model 8
Fitting model 9


In [37]:
from pyspark.sql import functions as fn

accuracies = [m. \
              transform(validation_df). \
              select(fn.avg(fn.expr('float(d_stars = prediction)')).alias('accuracy')). \
              first(). \
              accuracy for m in all_models]

In [38]:
accuracies

[0.7962320277640059,
 0.7962320277640059,
 0.7962320277640059,
 0.8170550322260783,
 0.8230044620723848,
 0.819533961328706,
 0.8215171046108082,
 0.8190381755081805,
 0.8081308874566188]

In [39]:
import numpy as np

In [40]:
best_model_idx = np.argmax(accuracies)
print("best model index =", best_model_idx)

best model index = 4


#### View the grid parameters of the best model

In [41]:
grid[best_model_idx]

{Param(parent='LogisticRegression_adde6b5d10b3', name='regParam', doc='regularization parameter (>= 0).'): 0.01,
 Param(parent='LogisticRegression_adde6b5d10b3', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.2}

In [42]:
best_model = all_models[best_model_idx]

In [43]:
accuracies[best_model_idx]

0.8230044620723848

In [44]:
# estimate generalization performance
best_model. \
    transform(test_df). \
    select(fn.avg(fn.expr('float(d_stars = prediction)')).alias('accuracy')). \
    show()

+------------------+
|          accuracy|
+------------------+
|0.8514376996805112|
+------------------+



Lets create a pipeline transformation by chaining the `idf_pipeline` with the logistic regression step (`lr`)

In [45]:
lr_pipeline = Pipeline(stages=[idf_pipeline, en_lr]).fit(train_df)

Lets estimate the accuracy:

In [46]:
lr_pipeline.transform(validation_df). \
            select(fn.expr('float(prediction = d_stars)').alias('correct')). \
            select(fn.avg('correct')).show()

+------------------+
|      avg(correct)|
+------------------+
|0.7992067426871592|
+------------------+

